In [ ]:
import sys
import platform
import os
import random as rnd
from math import sqrt, sin, cos, radians
import numpy as np
import pandas as pd
from numba import jit
import numba
import numexpr as ne
import matplotlib.pyplot as plt
import seaborn

In [ ]:
%matplotlib inline 
seaborn.set()

# Software Versions and Machine Hardware

In [ ]:
print("Python: {}\nNumpy: {}\nPandas: {}\nNumba: {}\nNumexpr: {}\n{}".format(sys.version,
                                                                             np.__version__,
                                                                             pd.__version__,
                                                                             numba.__version__,
                                                                             ne.__version__,
                                                                             ne.get_vml_version()))
print("Processor: {}\n# of Cores: {}\nMachine: {}\nArchitecture: {}".format(platform.processor(),
                                                                            os.cpu_count(),
                                                                            platform.machine(),
                                                                            platform.architecture()))
#Note:
#Intel64 Family 6 Model 78 Stepping 3 = Intel(R) Core(TM) i5-6300U CPU @ 2.40GHz

# Define Sample Data

List of tuples where list is made of tuples and each tuple has 4 values defining the point.

In [ ]:
points_10k = []
points_1m = []
size = (10000, 1000000)
points_obj = (points_10k, points_1m)
for s, var in ((size[0], points_obj[0]), (size[1], points_obj[1])):
    for i in range(s):
        var.append((rnd.uniform(-85,85),
                       rnd.uniform(-175,175),
                       rnd.uniform(-50,12000), 
                       rnd.uniform(-50,50)))
    print ('{} of {} containing {}, length: {}.'.format(type(var),type(var[0]),type(var[0][0]),len(var)))

Numpy arrays...

In [ ]:
#instead of using np.random.uniform to define these arrays, converting the points list
#so the same values are used for all methods
points_10k_np = np.empty([4,size[0]], dtype=np.double)
points_1m_np = np.empty([4,size[1]], dtype=np.double)
points_obj_np = (points_10k_np, points_1m_np)

for var in points_obj_np:
    for i, point in enumerate(var):
        var[:,i] = [point[0], point[1], point[2], point[3]]
    print ('{}, shape: {}.'.format(type(var),var.shape))

Pandas DataFrame of the points.

In [ ]:
points_10k_df = pd.DataFrame(points_10k, columns=['Lat','Long','Alt','Geoid'])
points_1m_df = pd.DataFrame(points_1m, columns=['Lat','Long','Alt','Geoid'])
points_obj_df = (points_10k_df, points_1m_df)
for var in points_obj_df:
    print ('{} containing {}, length: {}.'.format(type(var),
                                                  type(var.ix[0,0]),
                                                  len(var)))

In [ ]:
#timing_results = pd.DataFrame(columns=['Name','Loops','Repeat','Time (sec)','Num of Points'])
timing_results = pd.DataFrame()

def timing_results_filler (r, return_size=3):
    #TODO: remove return_size
    returner = {}#np.zeros(return_size)
#     if return_size < 3:
#         print ("error condition")
#         return returner
    #Loops,Repeat,Best
#     returner[0] = r.loops
#     returner[1] = r.repeat
#     returner[2] = r.best
    returner['Loops'] = r.loops
    returner['Repeat'] = r.repeat
    returner['Time (sec)'] = r.best
    return returner

# Constants

In [ ]:
a = 6378137  # Semi-major axis 
b = 6356752.3142  # Semi-minor axis
f = (a - b) / a  # flattening
e = 0.081819191 # eccentricity
ee = e*e
LOOPS = 50
REPEAT = 5

# Native Python Lists

In [ ]:
def LLAtoXYZ_raw (latitude, longitude, altitude, geoidSepIn = 0):
    # LLAtoXYZ converts a position (latitude, longitude, altitude) to the ECEF X, Y, Z format.
    # The function expects Lat and Long in degrees and Alt in meters and outputs X, Y, Z in meters. 
    # geoidSepIn is the height of the geoid above the WGS84 ellipsoid. It is optional and assumed 
    # to be 0 if not provided (meaning geoid-to-ellipsoid differences are ignored)
    
    latitude_rad = radians(latitude)
    longitude_rad = radians(longitude)
    height = altitude - geoidSepIn
    cos_lat = cos(latitude_rad)
     
    r = a / sqrt(1 - ee * sin(latitude_rad) * sin(latitude_rad))
    x = (r + height) * cos_lat * cos(longitude_rad)
    y = (r + height) * cos_lat * sin(longitude_rad)
    z = ((1 - ee) * r + height) * sin(latitude_rad)
        
    return x,y,z
LLAtoXYZ_raw(-38.123456,-124.65432,230,-20)
#X= -2856867.422762463
#Y= -4132876.8004144537
#Z= -3916387.577890978

In [ ]:
test_config_name = 'Native Python'
for points in points_obj:
    results = []
    r = %timeit -n 50 -r 5 -o for p in points: results.append(LLAtoXYZ_raw(p[0],p[1],p[2],p[3]))
    row = timing_results_filler(r)
    row['Name'] = test_config_name
    row['Num of Points'] = len(points)
    timing_results = timing_results.append(pd.Series(row),ignore_index=True)

# Numpy Arrays

In [ ]:
#import numpy functions directly
from numpy import sqrt, sin, cos, radians

In [ ]:
#def LLAtoXYZ_numpy (points_np, return_array = False):
def LLAtoXYZ_numpy (latitude, longitude, altitude, geoidSepIn = 0, return_array = False):
    # LLAtoXYZ converts a position (latitude, longitude, altitude) to the ECEF X, Y, Z format.
    # The function expects Lat and Long in degrees and Alt in meters and outputs X, Y, Z in meters. 
    # geoidSepIn is the height of the geoid above the WGS84 ellipsoid. It is optional and assumed 
    # to be 0 if not provided (meaning geoid-to-ellipsoid differences are ignored)
    
#    latitude = points_np[0]
#    longitude = points_np[1]
#    altitude = points_np[2]

#    if min(points_np.shape) == 4:
#        geoidSepIn = points_np[3]
#    else:
#        geoidSepIn = 0
    
    latitude_rad = radians(latitude)
    longitude_rad = radians(longitude)
    height = altitude - geoidSepIn
    cos_lat = cos(latitude_rad)
     
    r = a / sqrt(1 - ee * sin(latitude_rad) * sin(latitude_rad))
    x = (r + height) * cos_lat * cos(longitude_rad)
    y = (r + height) * cos_lat * sin(longitude_rad)
    z = ((1 - ee) * r + height) * sin(latitude_rad)
    
    if return_array:
        return np.array([x,y,z])
    else:
        return x,y,z
    
LLAtoXYZ_numpy(-38.123456,-124.65432,230,-20)
#X= -2856867.422762463
#Y= -4132876.8004144537
#Z= -3916387.577890978

In [ ]:
test_config_name = 'Numpy (Native)'
for points in points_obj_np:
    r = %timeit -n 50 -r 5 -o results = LLAtoXYZ_numpy(points[:,0],points[:,1],points[:,2],points[:,3],True)
    row = timing_results_filler(r)
    row['Name'] = test_config_name
    row['Num of Points'] = points.shape[1]
    timing_results = timing_results.append(pd.Series(row),ignore_index=True)

# Pandas (Serialized)
added to show how slow pandas can be if not vectorized

In [ ]:
# r = %timeit -n 5 -r 5 -o points_df['X'],points_df['Y'],points_df['Z'] = \ 
#                           zip(*points_df.apply(lambda row: LLAtoXYZ_raw(row[0], \
#                                                                         row[1], \
#                                                                         row[2], \
#                                                                         row[3]), \
#                                                                         axis=1))
# timing_results.loc['Pandas (Serialized)'] = timing_results_filler(r)

# Pandas (Vectorized)

In [ ]:
test_config_name = 'Pandas (Native)'
for points_df in points_obj_df:
    r = %timeit -n 50 -r 5 -o points_df['X'],points_df['Y'],points_df['Z'] = LLAtoXYZ_numpy(points_df['Lat'], \
                                                                                            points_df['Long'], \
                                                                                            points_df['Alt'], \
                                                                                            points_df['Geoid'],False)
    row = timing_results_filler(r)
    row['Name'] = test_config_name
    row['Num of Points'] = len(points_df)
    timing_results = timing_results.append(pd.Series(row),ignore_index=True)

# Numba

In [ ]:
@jit
def LLAtoXYZ_jit (latitude, longitude, altitude, geoidSepIn = 0, return_array = False):
    # LLAtoXYZ converts a position (latitude, longitude, altitude) to the ECEF X, Y, Z format.
    # The function expects Lat and Long in degrees and Alt in meters and outputs X, Y, Z in meters. 
    # geoidSepIn is the height of the geoid above the WGS84 ellipsoid. It is optional and assumed 
    # to be 0 if not provided (meaning geoid-to-ellipsoid differences are ignored)
    
    latitude_rad = radians(latitude)
    longitude_rad = radians(longitude)
    height = altitude - geoidSepIn
    cos_lat = cos(latitude_rad)
     
    r = a / np.sqrt(1 - ee * sin(latitude_rad) * sin(latitude_rad))
    x = (r + height) * cos_lat * cos(longitude_rad)
    y = (r + height) * cos_lat * sin(longitude_rad)
    z = ((1 - ee) * r + height) * sin(latitude_rad)
    
    if return_array:
        return np.array([x,y,z])
    else:
        return x,y,z
    
LLAtoXYZ_jit(-38.123456,-124.65432,230,-20)
#X= -2856867.422762463
#Y= -4132876.8004144537
#Z= -3916387.577890978

In [ ]:
test_config_name = 'Numba (with Numpy Arrays)'
for points in points_obj_np:
    r = %timeit -n 50 -r 5 -o results = LLAtoXYZ_jit(points[:,0],points[:,1],points[:,2],points[:,3],True)
    row = timing_results_filler(r)
    row['Name'] = test_config_name
    row['Num of Points'] = points.shape[1]
    timing_results = timing_results.append(pd.Series(row),ignore_index=True)

In [ ]:
test_config_name = 'Numba (with Pandas)'
for points_df in points_obj_df:
    r = %timeit -n 50 -r 5 -o points_df['X'],points_df['Y'],points_df['Z'] = LLAtoXYZ_jit(points_df['Lat'], \
                                                                                          points_df['Long'], \
                                                                                          points_df['Alt'], \
                                                                                          points_df['Geoid'],False)
    row = timing_results_filler(r)
    row['Name'] = test_config_name
    row['Num of Points'] = len(points_df)
    timing_results = timing_results.append(pd.Series(row),ignore_index=True)

# Numexpr

In [ ]:
def LLAtoXYZ_ne (latitude, longitude, altitude, geoidSepIn = 0, return_array = False):
    # LLAtoXYZ converts a position (latitude, longitude, altitude) to the ECEF X, Y, Z format.
    # The function expects Lat and Long in degrees and Alt in meters and outputs X, Y, Z in meters. 
    # geoidSepIn is the height of the geoid above the WGS84 ellipsoid. It is optional and assumed 
    # to be 0 if not provided (meaning geoid-to-ellipsoid differences are ignored)
    
    latitude_rad = radians(latitude)
    longitude_rad = radians(longitude)
    height = altitude - geoidSepIn
     
    r = ne.evaluate("a / sqrt(1 - ee * sin(latitude_rad)**2)")
    x = ne.evaluate("(r + height) * cos(latitude_rad) * cos(longitude_rad)")
    y = ne.evaluate("(r + height) * cos(latitude_rad) * sin(longitude_rad)")
    z = ne.evaluate("((1 - ee) * r + height) * sin(latitude_rad)")
    
    if return_array:
        return np.array([x,y,z])
    else:
        return x,y,z
    
LLAtoXYZ_ne(-38.123456,-124.65432,230,-20)
#X= -2856867.422762463
#Y= -4132876.8004144537
#Z= -3916387.577890978

In [ ]:
test_config_name = 'Numexpr (with Numpy Arrays)'
for points in points_obj_np:
    r = %timeit -n 50 -r 5 -o results = LLAtoXYZ_ne(points[:,0],points[:,0],points[:,0],points[:,0],True)
    row = timing_results_filler(r)
    row['Name'] = test_config_name
    row['Num of Points'] = points.shape[1]
    timing_results = timing_results.append(pd.Series(row),ignore_index=True)

In [ ]:
test_config_name = 'Numexpr (with Pandas)'
for points_df in points_obj_df:
    r = %timeit -n 50 -r 5 -o points_df['X'],points_df['Y'],points_df['Z'] = LLAtoXYZ_ne(points_df['Lat'], \
                                                                                         points_df['Long'], \
                                                                                         points_df['Alt'], \
                                                                                         points_df['Geoid'],False)
    row = timing_results_filler(r)
    row['Name'] = test_config_name
    row['Num of Points'] = len(points_df)
    timing_results = timing_results.append(pd.Series(row),ignore_index=True)

# Cython

In [ ]:
%load_ext Cython

In [ ]:
%%cython
from cython cimport boundscheck, wraparound
from libc.math cimport sin, cos, sqrt, M_PI
import numpy as np
cimport numpy as cnp

cdef double geoidSep = -29.701  # meters 

cdef double a = 6378137       # Semi-major axis 
cdef double b = 6356752.3142  # Semi-minor axis
cdef double f = (a - b) / a   # flattening
cdef double e = 0.081819191
cdef double ecc = e*e
cdef double radians = M_PI/180.0

@boundscheck(False)
@wraparound(False)
def LLAtoXYZ_cy(double[:] latitude, 
                double[:] longitude, 
                double[:] altitude, 
                double[:] geoidSepIn):
    cdef int i
    cdef int l = len(latitude)
    
    cdef double[:] x = np.empty(l, dtype=np.double)
    cdef double[:] y = np.empty(l, dtype=np.double)
    cdef double[:] z = np.empty(l, dtype=np.double)
    
    cdef double latitude_rad
    cdef double longitude_rad
    cdef double height
    cdef double s_lat
    cdef double c_lat
    cdef double r
    
    for i in range(l):
        latitude_rad = radians*latitude[i]
        longitude_rad = radians*longitude[i]
        height = altitude[i] - geoidSepIn[i]
        s_lat  = sin(latitude_rad)
        c_lat  = cos(latitude_rad)
         
        r = a / sqrt(1 - ecc * s_lat * s_lat)
        x[i] = (r + height) * c_lat * cos(longitude_rad)
        y[i] = (r + height) * c_lat * sin(longitude_rad)
        z[i] = ((1-ecc) * r + height) * s_lat
        
    return x,y,z

In [ ]:
results_c = LLAtoXYZ_cy(np.array([-38.123456], dtype=np.double),
                     np.array([-124.65432], dtype=np.double),
                     np.array([230], dtype=np.double),
                       np.array([-20], dtype=np.double))
print (results_c[0][0], results_c[1][0], results_c[2][0])
#X= -2856867.422762463
#Y= -4132876.8004144537
#Z= -3916387.577890978)

In [ ]:
test_config_name = 'Cython (with Numpy Arrays)'
for points in points_obj_np:
    r = %timeit -n 50 -r 5 -o results=LLAtoXYZ_cy(points[:,0],points[:,1],points[:,2],points[:,3])
    row = timing_results_filler(r)
    row['Name'] = test_config_name
    row['Num of Points'] = points.shape[1]
    timing_results = timing_results.append(pd.Series(row),ignore_index=True)

# Cython (parallel)

In [ ]:
%%cython
from cython cimport boundscheck, wraparound
from cython.parallel cimport prange
from libc.math cimport sin, cos, sqrt, M_PI
import numpy as np
cimport numpy as cnp

cdef double geoidSep = -29.701  # meters 

cdef double a = 6378137       # Semi-major axis 
cdef double b = 6356752.3142  # Semi-minor axis
cdef double f = (a - b) / a   # flattening
cdef double e = 0.081819191
cdef double ecc = e*e
cdef double radians = M_PI/180.0

@boundscheck(False)
@wraparound(False)
def LLAtoXYZ_cy_par(double[:] latitude, 
                double[:] longitude, 
                double[:] altitude, 
                double[:] geoidSepIn):
    cdef int i
    cdef int l = len(latitude)
    
    cdef double[:] x = np.empty(l, dtype=np.double)
    cdef double[:] y = np.empty(l, dtype=np.double)
    cdef double[:] z = np.empty(l, dtype=np.double)
    
    cdef double latitude_rad
    cdef double longitude_rad
    cdef double height
    cdef double s_lat
    cdef double c_lat
    cdef double r
    
    for i in prange(l, nogil=True):
        latitude_rad = radians*latitude[i]
        longitude_rad = radians*longitude[i]
        height = altitude[i] - geoidSepIn[i]
        s_lat  = sin(latitude_rad)
        c_lat  = cos(latitude_rad)
         
        r = a / sqrt(1 - ecc * s_lat * s_lat)
        x[i] = (r + height) * c_lat * cos(longitude_rad)
        y[i] = (r + height) * c_lat * sin(longitude_rad)
        z[i] = ((1-ecc) * r + height) * s_lat
        
    return x,y,z

In [ ]:
results_c = LLAtoXYZ_cy(np.array([-38.123456], dtype=np.double),
                     np.array([-124.65432], dtype=np.double),
                     np.array([230], dtype=np.double),
                       np.array([-20], dtype=np.double))
print (results_c[0][0], results_c[1][0], results_c[2][0])
#X= -2856867.422762463
#Y= -4132876.8004144537
#Z= -3916387.577890978)

In [ ]:
test_config_name = 'Cython.Parallel (with Numpy Arrays)'
for points in points_obj_np:
    r = %timeit -n 50 -r 5 -o results=LLAtoXYZ_cy_par(points[:,0],points[:,1],points[:,2],points[:,3])
    row = timing_results_filler(r)
    row['Name'] = test_config_name
    row['Num of Points'] = points.shape[1]
    timing_results = timing_results.append(pd.Series(row),ignore_index=True)

# Summary

In [ ]:
def results_update(data):
    data = data.set_index('Name')
    data['Vs Native Python'] = data.loc['Native Python']['Time (sec)'] / data['Time (sec)']
    data['Time per Point (usec)'] = data['Time (sec)'] / data['Num of Points'] * 1000000
    return data

In [ ]:
timing_results = timing_results.groupby('Num of Points').apply(results_update)

In [ ]:
timing_results

In [ ]:
timing_results.to_csv('results.csv')

Ran the time tests for both 10K and 1M points.

In [ ]:
tenK_df = timing_results.where(timing_results['Num of Points'] == 10000).dropna().reset_index(0,drop=True)
tenK_df.sort_values('Time (sec)', inplace=True, ascending=False)
tenK_df_subset = tenK_df.reindex(columns=['Time (sec)'])
ax_10k_time = tenK_df_subset.plot(kind='barh')
ax_10k_time.set_yticklabels(list(tenK_df.index))
ax_10k_time.set_xbound(upper=.004)
ax_10k_time.set_xlabel('seconds')
ax_10k_time.set_title('Total Time for 10K Points by Method')
plt.savefig('time_10k.png')

In [ ]:
tenK_df.sort_values('Vs Native Python', inplace=True, ascending=True)
tenK_df_subset = tenK_df.reindex(columns=['Vs Native Python'])
ax_10k_vs = tenK_df_subset.plot(kind='barh')
ax_10k_vs.set_yticklabels(list(tenK_df.index))
ax_10k_vs.set_xlabel('Multiplier')
ax_10k_vs.set_title('Method Compared to Native Python (10K Points)')
plt.savefig('compare_10k.png')

In [ ]:
oneM_df = timing_results.where(timing_results['Num of Points'] == 1000000).dropna().reset_index(0,drop=True)
oneM_df.sort_values('Time (sec)', inplace=True, ascending=False)
oneM_df_subset = oneM_df.reindex(columns=['Time (sec)'])
ax_1M_time = oneM_df_subset.plot(kind='barh')
ax_1M_time.set_yticklabels(list(oneM_df.index))
ax_1M_time.set_xbound(upper=.25)
ax_1M_time.set_xlabel('seconds')
ax_1M_time.set_title('Total Time for 1M Points by Method')
plt.savefig('time_1M.png')

In [ ]:
oneM_df.sort_values('Vs Native Python', inplace=True, ascending=True)
oneM_df_subset = oneM_df.reindex(columns=['Vs Native Python'])
ax_1M_vs = oneM_df_subset.plot(kind='barh')
ax_1M_vs.set_yticklabels(list(oneM_df.index))
ax_1M_vs.set_xlabel('Multiplier')
ax_1M_vs.set_title('Method Compared to Native Python (1M Points)')
plt.savefig('compare_1M.png')

In [ ]:
oneM_per_pt = pd.Series(oneM_df['Time per Point (usec)'],name='1M Points')
tenK_per_pt = pd.Series(tenK_df['Time per Point (usec)'],name='10K Points')
oneM_per_pt = oneM_per_pt * 1000
tenK_per_pt = tenK_per_pt * 1000
per_pt_df = pd.concat((oneM_per_pt, tenK_per_pt),axis=1)
per_pt_df.sort_values('1M Points', inplace=True, ascending=False)
ax_per_pt = per_pt_df.plot(kind='barh', width = .8, figsize=(12,7))
ax_per_pt.set_xbound(upper=500)
ax_per_pt.set_title('Per Point Execution Time', fontsize=18)
ax_per_pt.set_xlabel('nanoseconds', fontsize=18)
ax_per_pt.xaxis.grid(True, linestyle='--', which='major', color='grey', alpha=.5)
plt.legend(fontsize=14)
rects = ax_per_pt.patches
labels = per_pt_df.values.flatten('F')
ax_per_pt.tick_params(axis='y', labelsize=14)
for rect, label in zip(rects, labels):
    width = rect.get_width()
    height = rect.get_height()
    ax_per_pt.text(min(width - 20, 475), rect.get_y()+ height*3/4, int(label), color='white',fontsize=12, ha='left', va='top', weight='heavy')
plt.savefig('exec_time.png')

In [ ]:
oneM_vs = pd.Series(oneM_df['Vs Native Python'],name='1M Points')
tenK_vs = pd.Series(tenK_df['Vs Native Python'],name='10K Points')
vs_df = pd.concat((oneM_vs, tenK_vs),axis=1)
vs_df.sort_values('1M Points', inplace=True, ascending=False)
ax_vs = vs_df.plot(kind='barh', width = .8, figsize=(12,7))
ax_vs.set_title('Method Compared to Native Python', fontsize=18)
ax_vs.set_xlabel('Improvement Over Native Python (as Multiplier)', fontsize=18)
ax_vs.xaxis.grid(True, linestyle='--', which='major', color='grey', alpha=.5)
plt.legend(fontsize=14)
rects = ax_vs.patches
labels = vs_df.values.flatten('F')
labels = np.rint(labels)
ax_vs.tick_params(axis='y', labelsize=14)
for rect, label in zip(rects, labels):
    width = rect.get_width()
    ax_vs.text(width + 50, rect.get_y() + height/2, int(label), 
               fontsize=12, ha='left', va='center', weight='heavy')
plt.savefig('method_compare.png')